### Importing the Libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

import re #regular expessions

! pip install beautifulsoup4
! pip install requests
import requests #request online data
from bs4 import BeautifulSoup as bs # library for navigating html files

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.52         |     pyhd8ed1ab_0          35 KB  conda-forge
    geopy-2.2.0                |     pyhd8ed1ab_0          67 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         102 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.52-pyhd8ed1ab_0
  geopy              conda-forge/noarch::geopy-2.2.0-pyhd8ed1ab_0



geographiclib-1.52   | 35 KB     | ##################################### | 100% 
geopy-2.2.0          | 67 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transa

### Combining for Clustering

In [6]:
table_one = pd.read_csv('atlanta_clustering_category.csv')
table_two = pd.read_csv('dallas_clustering_category.csv')
table_three = pd.read_csv('seattle_clustering_category.csv')
table_four = pd.read_csv('newyork_clustering_category.csv')

In [7]:
table_one = table_one.drop('Unnamed: 0', axis = 1)
print(len(table_one))
table_one.head()

134


,Venue_Distance,Category_ID,Category_Name,Venue_Rating
0,6,4bf58dd8d48988d174941735,Coworking Space,Not Rated
1,20,4bf58dd8d48988d1f1931735,General Entertainment,Not Rated
2,20,5032885091d4c4b30a586d66,Real Estate Office,Not Rated
3,23,4d954b06a243a5684965b473,Residential Building (Apartment / Condo),Not Rated
4,23,5032885091d4c4b30a586d66,Real Estate Office,Not Rated


In [8]:
table_two = table_two.drop('Unnamed: 0', axis = 1)
print(len(table_two))
table_two.head()

154


,Venue_Distance,Category_ID,Category_Name,Venue_Rating
0,6,4bf58dd8d48988d1f2931735,Performing Arts Venue,8.2
1,10,4bf58dd8d48988d1e7931735,Jazz Club,8.2
2,23,4bf58dd8d48988d1c1941735,Mexican Restaurant,8.0
3,46,4bf58dd8d48988d1e4931735,Bowling Alley,Not Rated
4,60,4bf58dd8d48988d1e4931735,Bowling Alley,Not Rated


In [9]:
table_three = table_three.drop('Unnamed: 0', axis = 1)
print(len(table_three))
table_three.head()

286


,Venue_Distance,Category_ID,Category_Name,Venue_Rating
0,5,4bf58dd8d48988d1c9941735,Ice Cream Shop,Not Rated
1,5,4bf58dd8d48988d1de931735,Tattoo Parlor,7.7
2,7,55a59bace4b013909087cb2a,Udon Restaurant,7.7
3,13,4bf58dd8d48988d1df931735,BBQ Joint,Not Rated
4,16,4bf58dd8d48988d16c941735,Burger Joint,Not Rated


In [10]:
table_four = table_four.drop('Unnamed: 0', axis = 1)
print(len(table_four))
table_four.head()

755


,Venue_Distance,Category_ID,Category_Name,Venue_Rating
0,3,4bf58dd8d48988d124941735,Office,Not Rated
1,7,4bf58dd8d48988d124941735,Office,Not Rated
2,11,4bf58dd8d48988d105941735,Gym Pool,Not Rated
3,15,4bf58dd8d48988d176941735,Gym,7.6
4,24,4bf58dd8d48988d151941735,Taco Place,Not Rated


In [13]:
combined_table = table_one.append(table_two)
combined_table = combined_table.append(table_three)
combined_table = combined_table.append(table_four)
print(len(combined_table))
combined_table.head()

1329


,Venue_Distance,Category_ID,Category_Name,Venue_Rating
0,6,4bf58dd8d48988d174941735,Coworking Space,Not Rated
1,20,4bf58dd8d48988d1f1931735,General Entertainment,Not Rated
2,20,5032885091d4c4b30a586d66,Real Estate Office,Not Rated
3,23,4d954b06a243a5684965b473,Residential Building (Apartment / Condo),Not Rated
4,23,5032885091d4c4b30a586d66,Real Estate Office,Not Rated


In [14]:
combined_table = combined_table.sort_values(['Venue_Rating', 'Category_ID','Venue_Distance'], ascending = False)
combined_table.head()

,Venue_Distance,Category_ID,Category_Name,Venue_Rating
159,11,5f2c2834b6d05514c704451e,Escape Room,Not Rated
40,20,5bae9231bedf3950379f89d4,Poke Place,Not Rated
718,49,58daa1558bbb0b01f18ec200,Cooking School,Not Rated
27,61,5744ccdfe4b0c0459246b4c7,Child Care Service,Not Rated
270,33,5744ccdfe4b0c0459246b4c4,Public Bathroom,Not Rated


In [16]:
combined_table= combined_table.set_index('Category_Name')

In [17]:
combined_table.head()

,Venue_Distance,Category_ID,Venue_Rating
Category_Name,,,
Escape Room,11,5f2c2834b6d05514c704451e,Not Rated
Poke Place,20,5bae9231bedf3950379f89d4,Not Rated
Cooking School,49,58daa1558bbb0b01f18ec200,Not Rated
Child Care Service,61,5744ccdfe4b0c0459246b4c7,Not Rated
Public Bathroom,33,5744ccdfe4b0c0459246b4c4,Not Rated


In [18]:
combined_table.to_csv('combined_table.csv')

### Combining Cities

### grouping

In [53]:
combined_table = pd.read_csv('combined_table.csv')
print(len(combined_table))
combined_table.head()

1329


,Category_Name,Venue_Distance,Category_ID,Venue_Rating
0,Escape Room,11,5f2c2834b6d05514c704451e,Not Rated
1,Poke Place,20,5bae9231bedf3950379f89d4,Not Rated
2,Cooking School,49,58daa1558bbb0b01f18ec200,Not Rated
3,Child Care Service,61,5744ccdfe4b0c0459246b4c7,Not Rated
4,Public Bathroom,33,5744ccdfe4b0c0459246b4c4,Not Rated


In [54]:
combined_table = combined_table.drop('Category_Name', axis = 1)
combined_table['Venue_Rating'].mask(combined_table['Venue_Rating']=='Not Rated', 0, inplace = True)
combined_table['Venue_Rating'] = combined_table['Venue_Rating'].astype('float')
combined_table.head()

,Venue_Distance,Category_ID,Venue_Rating
0,11,5f2c2834b6d05514c704451e,0.0
1,20,5bae9231bedf3950379f89d4,0.0
2,49,58daa1558bbb0b01f18ec200,0.0
3,61,5744ccdfe4b0c0459246b4c7,0.0
4,33,5744ccdfe4b0c0459246b4c4,0.0


In [56]:
vrvd_grouped = combined_table.groupby('Category_ID').mean()
vrvd_grouped.head()

,Venue_Distance,Venue_Rating
Category_ID,,
4bf58dd8d48988d101941735,132.666667,0.0
4bf58dd8d48988d101951735,92.000000,0.0
4bf58dd8d48988d102941735,59.400000,0.0
4bf58dd8d48988d102951735,27.000000,0.0
4bf58dd8d48988d103951735,126.777778,0.6


In [17]:
ci_grouped = combined_table.groupby('Category_ID').count()
ci_grouped = ci_grouped.drop('Venue_Rating', axis = 1)
ci_grouped = ci_grouped.rename(columns ={'Venue_Distance':'Category_Count'})
print(ci_grouped.shape)
ci_grouped.head()

(309, 1)


,Category_Count
Category_ID,
4bf58dd8d48988d101941735,3
4bf58dd8d48988d101951735,2
4bf58dd8d48988d102941735,5
4bf58dd8d48988d102951735,1
4bf58dd8d48988d103951735,9


In [61]:
all_grouped = ci_grouped.join(vrvd_grouped)
print(all_grouped.shape)
all_grouped.head()

(309, 3)


,Category_Count,Venue_Distance,Venue_Rating
Category_ID,,,
4bf58dd8d48988d101941735,3,132.666667,0.0
4bf58dd8d48988d101951735,2,92.000000,0.0
4bf58dd8d48988d102941735,5,59.400000,0.0
4bf58dd8d48988d102951735,1,27.000000,0.0
4bf58dd8d48988d103951735,9,126.777778,0.6


In [64]:
all_grouped.sort_values('Category_Count')
#either need to drop Not Rated values or normalize total after making zeros

,Category_Count,Venue_Distance,Venue_Rating
Category_ID,,,
4bf58dd8d48988d1d6941735,1,482.000000,0.000000
4bf58dd8d48988d1f0931735,1,1195.000000,0.000000
4bf58dd8d48988d1f1941735,1,108.000000,7.700000
4bf58dd8d48988d1f4931735,1,1185.000000,0.000000
4bf58dd8d48988d1f5941735,1,35.000000,0.000000
4bf58dd8d48988d1f9941735,1,60.000000,0.000000
4bf58dd8d48988d1fa941735,1,179.000000,0.000000
4bf58dd8d48988d1fe941735,1,13.000000,0.000000
4c2cd86ed066bed06c3c5209,1,421.000000,0.000000
